In [1]:
import pandas as pd
import pysam
import numpy as np
import sys
from collections import defaultdict
import time
start_time = time.time()


bamfile='/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/softreadcounting/testtheidea/veryefficient/fulltest/CD4allrange_NR_1000000_insilmix45_sorted'


#####sm must be hypo###

sm='/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/softreadcounting/testtheidea/veryefficient/BL14_all_matrixCin_nr0.4_imputed_rowmean.txt_bg_intesectedwith_CD4DMRofBL14atleast.2SM_g1_CD4_3_g2_others_33_withheader.txt_othermean.txt'


mapping_quality=40
base_quality=20


deltagreaterforpositive=0  ##############>0

deltasmallerfornegative=0 ############<0



In [2]:
smdf=pd.read_csv(sm,sep='\t',index_col=['chrom','start'])

smdf.head()
   

CD4-others  CD8-others  nB-others  NK-others  Mn-others  \
chrom start                                                               
chr1  32387132   -0.707256   -0.392077   0.130231   0.104744   0.105103   
      32387376   -0.716974   -0.172051   0.165744   0.095385   0.223179   
      32387403   -0.783192    0.005833   0.101679   0.091269   0.146551   
      32387409   -0.764192    0.083705   0.105244   0.106321   0.097346   
      32387417   -0.295179    0.017846   0.042615   0.016769   0.063795   

                mNeu-others  m8-others  DC-others  Eo-others  Tr-others  \
chrom start                                                               
chr1  32387132     0.101872   0.076385   0.118385   0.094154  -0.001692   
      32387376     0.072410   0.040103   0.145641  -0.019128  -0.107436   
      32387403     0.096654  -0.000449   0.146551   0.113167  -0.093603   
      32387409     0.070423   0.105603   0.116372   0.116372  -0.145321   
      32387417    -0.030256   0.063795   0.043692   0.027718  -0.011590   

                m4-others  Mg-others  Er-others  mB-others  
chrom start                                                 
chr1  32387132   0.052692   0.138846   0.054846   0.123769  
      32387376  -0.061128   0.113872   0.109564   0.110821  
      32387403  -0.164141   0.077090   0.132551   0.130038  
      32387409  -0.155013   0.093218   0.103449   0.066474  
      32387417  -0.032590   0.023949   0.040103   0.029333

In [3]:
scorecolumns=smdf.columns
scoredict=defaultdict(list)

In [4]:
def populate_score_matrix(aRead,direction):
    Refstart=aRead.reference_start
    

    Refend=aRead.reference_end-1
    

    Refchrom=aRead.reference_name
    

    
    
    
    corressm=smdf.loc[(Refchrom,Refstart):(Refchrom,Refend)]
    
    if corressm.shape[0]==0:
        return
    
    Readname=aRead.query_name
    readbase_qualities=aRead.query_qualities
    readbases=aRead.query_sequence
    refpositions=aRead.get_reference_positions(full_length=True)
    
    len_readbases=len(readbases)
    
    if len(readbase_qualities)!=len_readbases:
        print('len_readbase_qualities!=len_readbases')
        sys.exit(1)

    if len(refpositions)!=len_readbases:
        print("something wrong")
        print(aRead)
        print(Refend)
        print(Refstart)
        print(len(readbases))
        print(aRead.get_reference_positions(full_length=True))
        
        print(aRead.get_aligned_pairs(with_seq=True))
        
        sys.exit(1)
        
    for cindex, crow in corressm.iterrows():
        
    
        try:
            baseposindex=refpositions.index(cindex[1]+direction)
        except:
            continue
      
        

        
        
        readbase=readbases[baseposindex]
        if readbase==None:
            continue
        
        
        if readbase_qualities[baseposindex]<base_quality:
            continue
        
        readbase=readbase.upper()
        
        accpeted=0
        
        
        if direction==0:
            if readbase=='T':
                accpeted=-1
           # elif readbase=='C':
            #    accpeted=1
                
        elif direction==1:
            if readbase=='A':
                accpeted=-1
           # elif readbase=='G':
            #    accpeted=1
        
        if accpeted!=0 :
            
            smrow=crow.to_numpy()
            
            
            if len(scoredict[Readname])==0:
                    scoredict[Readname]=list(accpeted*smrow)
            else:
                scoredict[Readname]=list(np.array(scoredict[Readname])+(accpeted*smrow))
            
            
                
                
            

            
        
      
    
    
    
    
  
    
    
 
        
    

In [5]:
bam=pysam.AlignmentFile(bamfile, 'rb')

for read in bam.fetch():
    if read.mapping_quality < mapping_quality:
        continue
    if read.is_duplicate == True:
        continue
    
    offset = 0

    

    if read.has_tag('YD'):
        yd = read.get_tag("YD")
        if yd == "r":
            offset = 1

    elif read.has_tag('XB'):
        xb=read.get_tag("XB")

        if xb=='G':
            offset=1
        elif xb=='C':
            pass
        else:
            print("unknown xb value. Exiting....")
            print(read.query_name)

            sys.exit(1)

    else:
        print("no tag found. Exiting")
        print(read.query_name)
        sys.exit(1)
        
        
    
    


    populate_score_matrix(read,offset)
    

In [6]:
scoredf=pd.DataFrame.from_dict(scoredict, orient='index',columns=scorecolumns)
scoredf.head()

CD4-others  CD8-others  \
CD8.HWI-ST537_177:2:1112:11832:4142#2           0.707256    0.392077   
CD8.HWI-ST537_205:8:1207:20921:3430#19          1.414513    0.784154   
DC.HWI-ST539:218:C47JJACXX:6:2308:16035:2426    0.707256    0.392077   
DC.HWI-ST537:224:C4E15ACXX:1:1305:5682:93093    0.707256    0.392077   
Er.HWI-ST1450_141:8:1213:18765:81963#9          0.707256    0.392077   

                                              nB-others  NK-others  Mn-others  \
CD8.HWI-ST537_177:2:1112:11832:4142#2         -0.130231  -0.104744  -0.105103   
CD8.HWI-ST537_205:8:1207:20921:3430#19        -0.260462  -0.209487  -0.210205   
DC.HWI-ST539:218:C47JJACXX:6:2308:16035:2426  -0.130231  -0.104744  -0.105103   
DC.HWI-ST537:224:C4E15ACXX:1:1305:5682:93093  -0.130231  -0.104744  -0.105103   
Er.HWI-ST1450_141:8:1213:18765:81963#9        -0.130231  -0.104744  -0.105103   

                                              mNeu-others  m8-others  \
CD8.HWI-ST537_177:2:1112:11832:4142#2           -0.101872  -0.076385   
CD8.HWI-ST537_205:8:1207:20921:3430#19          -0.203744  -0.152769   
DC.HWI-ST539:218:C47JJACXX:6:2308:16035:2426    -0.101872  -0.076385   
DC.HWI-ST537:224:C4E15ACXX:1:1305:5682:93093    -0.101872  -0.076385   
Er.HWI-ST1450_141:8:1213:18765:81963#9          -0.101872  -0.076385   

                                              DC-others  Eo-others  Tr-others  \
CD8.HWI-ST537_177:2:1112:11832:4142#2         -0.118385  -0.094154   0.001692   
CD8.HWI-ST537_205:8:1207:20921:3430#19        -0.236769  -0.188308   0.003385   
DC.HWI-ST539:218:C47JJACXX:6:2308:16035:2426  -0.118385  -0.094154   0.001692   
DC.HWI-ST537:224:C4E15ACXX:1:1305:5682:93093  -0.118385  -0.094154   0.001692   
Er.HWI-ST1450_141:8:1213:18765:81963#9        -0.118385  -0.094154   0.001692   

                                              m4-others  Mg-others  Er-others  \
CD8.HWI-ST537_177:2:1112:11832:4142#2         -0.052692  -0.138846  -0.054846   
CD8.HWI-ST537_205:8:1207:20921:3430#19        -0.105385  -0.277692  -0.109692   
DC.HWI-ST539:218:C47JJACXX:6:2308:16035:2426  -0.052692  -0.138846  -0.054846   
DC.HWI-ST537:224:C4E15ACXX:1:1305:5682:93093  -0.052692  -0.138846  -0.054846   
Er.HWI-ST1450_141:8:1213:18765:81963#9        -0.052692  -0.138846  -0.054846   

                                              mB-others  
CD8.HWI-ST537_177:2:1112:11832:4142#2         -0.123769  
CD8.HWI-ST537_205:8:1207:20921:3430#19        -0.247538  
DC.HWI-ST539:218:C47JJACXX:6:2308:16035:2426  -0.123769  
DC.HWI-ST537:224:C4E15ACXX:1:1305:5682:93093  -0.123769  
Er.HWI-ST1450_141:8:1213:18765:81963#9        -0.123769

In [7]:
end_time = time.time()

time_elapsed = (end_time - start_time)

print(time_elapsed)

286.94085597991943


In [8]:
scoredf.to_csv(bamfile+"_scored_nohyper.txt",sep="\t")

In [9]:
ctpropdict= defaultdict(list)

for score in scorecolumns:
    

    
    posscore=scoredf.loc[scoredf[score]>deltagreaterforpositive,score]
    
    negscore=scoredf.loc[scoredf[score]<deltasmallerfornegative,score]
    
    
    poslen=len(posscore)
    neglen=len(negscore)
    
    posscoreSUM=sum(posscore)
    
    
    negscoreSUM=sum(negscore)
    
    
    
    ct_posnegscore=posscoreSUM #/(posscoreSUM+abs(negscoreSUM))
    
    
    ct_fragscore=poslen #/(poslen+neglen)
    
    
    
    
    
    ctname=score.replace('-others','')
    
    ctpropdict[ctname+"_posnegscore"].append(ct_posnegscore)

    ctpropdict[ctname+"_fragscore"].append(ct_fragscore)
    
    
    

In [10]:
#print(posscore)

In [11]:
ctpropdf=pd.DataFrame.from_dict(ctpropdict)
ctpropdf.to_csv(bamfile+"_softRD_nohyper_result.txt",sep="\t",index=False)

In [12]:
end_time = time.time()

time_elapsed = (end_time - start_time)

print(time_elapsed)

287.0664417743683
